In [23]:
%load_ext autoreload
%autoreload 2
import numpy as np
import enum
import copy
import connect4.connect4 as game
from pympler import asizeof
import deeplearning.buffer as bf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
env = game.Connect4()
p1 = game.RandomPlayer()
p2 = game.RandomPlayer()
buffer = bf.ReplayBuffer(1000)
gm = game.GameManager([p1, p2])

In [37]:
gm.play(1000, env, buffer)

In [32]:
buffer.create_training_examples(10)

([tensor([-0., -0., -0., -0., -0., -0., -0., -0., -0., -1., -0., -0., -0., -0.,
          -1.,  1., -1., -0., -0.,  1., -0.,  1., -1., -1., -0., -0., -1., -1.,
           1., -1.,  1., -0.,  1., -1.,  1., -1.,  1.,  1.,  1., -1., -1.,  1.,
          -1.]),
  tensor([-0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
          -0., -0., -1.,  1., -0., -1., -0.,  1.,  1.,  1., -1., -0., -1., -1.,
          -1., -1.,  1.,  1., -1., -1.,  1.,  1., -1., -1.,  1.,  1.,  1., -1.,
          -1.]),
  tensor([-0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
          -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
          -1., -0., -0., -0., -0., -0., -0.,  1., -1., -1., -0., -0.,  1., -0.,
          -1.]),
  tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  1., -1., -1.,  0.,